### **Objective**
You are provided with a dataset of images of plant seedlings at various stages of grown. Each image has a filename that is its unique id. The dataset comprises 12 plant species. The goal of the project is to create a classifier capable of determining a plant's species from a photo.

#### **Dataset:**

The data file names are:

- images.npy - which contains the array of images
- Label.csv  - Lables for each image 

In [1]:
#Reading the training images from the path and labelling them into the given categories
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import seaborn as sns # for data visualization 
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential #sequential api for sequential model 
from tensorflow.keras.layers import Dense, Dropout, Flatten #importing different layers 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Input, LeakyReLU,Activation
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical #to perform one-hot encoding 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam #optimiers for optimizing the model
from tensorflow.keras.callbacks import EarlyStopping  #regularization method to prevent the overfitting
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import losses, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Setup Done")

Setup Done


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

### **Reading the DataSet**

In [ ]:
# Read the data file 
images=np.load('/content/drive/MyDrive/Colab Notebooks/images.npy')
plant_seedlings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Labels.csv')
#images=np.load('images.npy')
#plant_seedlings = pd.read_csv('Labels.csv')
labels = plant_seedlings['Label']
# get #of classes
classes = list(np.unique(labels))
num_of_seedlings = len(images)
print("There are %d classes and %d images in the dataset" % (len(classes),len(images)))

In [ ]:
print("Indexed list of classes")
_=[print(f'{x} - {classes[x]}') for x in range(len(classes))]


In [ ]:
print("Percent wise distribution of each class")
print("-------------------------------------------")
print("{}".format(plant_seedlings['Label'].value_counts(normalize=True)*100))

In [ ]:
print("Count wise distribution of each class")
print("-------------------------------------------")
print("{}".format(plant_seedlings['Label'].value_counts(sort=True)))

### Observations:
There is unbalanced distribution of classes, Highest is *Loose Silky-ben*  with 13.76% and lowest is *Maize* with 4.65%

In [ ]:
# Build List of List containing HSV image and numberical class value 
seedlings_data = []
grayhsv_data =[]
scale = 128
for image,label in (list(zip(images,labels))):
  try:  
    image=cv2.resize(image,(scale,scale))
    grayimg=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    blurr = cv2.GaussianBlur(image,(5,5),0)
    #blurr = cv2.GaussianBlur(grayimg,(5,5),0)
    hsv = cv2.cvtColor(blurr,cv2.COLOR_BGR2HSV)
    #grayhsv = cv2.cvtColor(hsv,cv2.COLOR_BGR2GRAY)
    class_num=classes.index(label)
    seedlings_data.append([(image,hsv),class_num])
    #grayhsv_data.append([(grayimg,grayhsv),class_num])
  except:
    pass


In [ ]:
seedlings_data[0]

### **Exploratory Data Analysis**

In [ ]:
fig1, ax1 = plt.subplots(figsize=(8, 8))
ax1=plt.pie(plant_seedlings['Label'].value_counts(), labels=classes,shadow=True, autopct = lambda pct: '{:.2f}%'.format(pct),startangle=90)
plt.title("Proportion of seedlings in the imageset ", size = 20)
plt.show()


#### **Print Random Images from each class**

**Visualize images randomly from each of the three classes**. The Image matrix is plotted and each row represents two 3 channel images corresponding to one class.

In [ ]:

imgx_0 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 0] 
labelx_0 = classes[0]
imgx_1 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 1] 
labelx_1 = classes[1]
imgx_2 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 2] 
labelx_2 = classes[2]

imgx_3 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 3] 
labelx_3 = classes[3]
imgx_4 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 4] 
labelx_4 = classes[4]
imgx_5 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 5] 
labelx_5 = classes[5]

imgx_6 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 6] 
labelx_6 = classes[6]
imgx_7 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 7] 
labelx_7 = classes[7]
imgx_8 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 8] 
labelx_8 = classes[8]

imgx_9 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 9] 
labelx_9 = classes[9]
imgx_10 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 10] 
labelx_10 = classes[10]
imgx_11 = [imgs[0] for imgs,clsnum in seedlings_data if clsnum == 11] 
labelx_11 = classes[11]

In [ ]:
from tensorflow.keras.preprocessing import image
fig = plt.figure(figsize = (20,9))

for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+1)
    fn=image.array_to_img(imgx_0[np.random.randint(0,len(imgx_0))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_0)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+3)
    fn=image.array_to_img(imgx_1[np.random.randint(0,len(imgx_1))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_1)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+5)
    fn=image.array_to_img(imgx_2[np.random.randint(0,len(imgx_2))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_2)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+7)
    fn=image.array_to_img(imgx_3[np.random.randint(0,len(imgx_3))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_3)
    plt.axis('off')

for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+9)
    fn=image.array_to_img(imgx_4[np.random.randint(0,len(imgx_4))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_4)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+11)
    fn=image.array_to_img(imgx_5[np.random.randint(0,len(imgx_5))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_5)
    plt.axis('off')
    

for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+13)
    fn=image.array_to_img(imgx_6[np.random.randint(0,len(imgx_6))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_6)
    plt.axis('off')

for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+15)
    fn=image.array_to_img(imgx_7[np.random.randint(0,len(imgx_7))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_7)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+17)
    fn=image.array_to_img(imgx_8[np.random.randint(0,len(imgx_8))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_8)
    plt.axis('off')

for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+19)
    fn=image.array_to_img(imgx_9[np.random.randint(0,len(imgx_9))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_9)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+21)
    fn=image.array_to_img(imgx_10[np.random.randint(0,len(imgx_10))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_10)
    plt.axis('off')
    
for count in range(0,2):
    ax = fig.add_subplot(4, 6, count+23)
    fn=image.array_to_img(imgx_11[np.random.randint(0,len(imgx_11))])
    plt.imshow(fn, cmap='gray')
    plt.title(labelx_11)
    plt.axis('off')
plt.show()

#### Image Histogram
- Image Histograms shows the distribution on Pixel/Pixel Intensity in the images
- Histogram of an image provides a global description of the appearance of an image.
- Histogram of an image represents the relative frequency of occurence of various gray levels in an image.

In [ ]:
fig = plt.figure(figsize = (8,4))
i=seedlings_data[12][0][0]
fig.add_subplot(1,2,1)
plt.imshow(i)
plt.axis('off')
b=i.flatten()
#plt.figure(figsize=(4,4), edgecolor='red')
fig.add_subplot(1,2,2)
n, bins, patches = plt.hist(b, bins=15)
plt.xlabel('Pixel value')
plt.ylabel('Number of Pixels')
plt.title('Histogram of Pixel values')
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,4))
i=seedlings_data[12][0][1]
fig.add_subplot(1,2,1)
plt.imshow(i)
plt.axis('off')
b=i.flatten()
#plt.figure(figsize=(4,4), edgecolor='red')
fig.add_subplot(1,2,2)
n, bins, patches = plt.hist(b, bins=15)
plt.xlabel('Pixel value')
plt.ylabel('Number of Pixels')
plt.title('Histogram of Pixel values')
plt.show()

#### Observations:
There is case of class imbalance, the % of observations ranges from 13.7% as  high to 4.65% as low , this can bias the prediction towards majority class 

Sample Printing of images shows that the quality of images not very good and need to be cleaned up to remove the noise, which can be achieved by 

- Blurring  the images with Gaussian fliter
- Convert the RGB images into the HSV
- Histogram show higher pixed intensity due to HSV format conversion 
- This may help identify boudries 


**Finding the mean images** for each class of seedlings

In [ ]:

s_data = []
IMG_SIZE=128
for image,label in (list(zip(images,labels))):
    class_n = classes.index(label)
    if class_n == 0:
     img_array = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
     new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) 
     
     s_data.append([new_array])


In [ ]:
def get_data(class_d):
  sz_data = []
  IMG_SIZE=128
  for image,label in (list(zip(images,labels))):
    class_n = classes.index(label)
    if class_n == class_d:
     img_array = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
     new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE)) 
     sz_data.append([new_array])
  return sz_data

In [ ]:
def find_mean_img(full_mat, title):
    # calculate the average
    mean_img = np.mean(full_mat, axis = 0)
    mean_img = mean_img/255
    # reshape it back to a matrix
    #mean_img = mean_img.reshape((128,128,3))
    #plt.imshow(mean_img, vmin=0, vmax=255, cmap='Greys_r')
    plt.imshow(mean_img[0],  cmap='Greys_r')
    plt.title(f'Average {title}')
    plt.axis('off')
    plt.show()

In [ ]:
fig=plt.figure(figsize=(2,2))
label_0_data=get_data(0)
find_mean_img(label_0_data, classes[0])

label_1_data=get_data(1)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_1_data, classes[1])

label_2_data=get_data(2)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_2_data, classes[2])

label_3_data=get_data(3)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_3_data, classes[3])

In [ ]:
label_4_data=get_data(4)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_4_data, classes[4])

label_5_data=get_data(5)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_5_data, classes[5])

label_6_data=get_data(6)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_6_data, classes[6])

label_7_data=get_data(7)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_7_data, classes[7])

In [ ]:
label_8_data=get_data(8)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_8_data, classes[8])

label_9_data=get_data(9)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_9_data, classes[9])

label_10_data=get_data(10)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_10_data, classes[10])

label_11_data=get_data(11)
fig=plt.figure(figsize=(2,2))
find_mean_img(label_11_data, classes[11])

### **Data Preprocessing** 
Create X features(Images) and y lables from the seedlings data, we shuffle it random so we will get mix of images 
As observed the quality of images are very poor and also sample distribution in not proper 


In [ ]:
#Create X and y feature sets orgX is original image sets X is blurred normalize hsv sr
X = []
y = []
orgX =[]
np.random.shuffle(seedlings_data)
for features,label in seedlings_data:
#np.random.shuffle(grayhsv_data)
#for features,label in grayhsv_data:
    orgX.append(features[0])
    X.append(features[1])
    y.append(label)
X= np.array(X)
orgX = np.array(orgX)

# Normalizing pixel values  
X = X/255.0  
orgX = orgX/255.0
# image reshaping 
X = X.reshape(-1,128,128,3)
orgX = orgX.reshape(-1,128,128,3)
print(X.shape)
print(orgX.shape)

In [ ]:
# Split the data into train and test data set 
seed=42
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=seed,stratify=y) 

In [ ]:
#convert target to encoded categorical format 
encoded = to_categorical(np.array(y_train))
y_train_e=encoded
encoded_test = to_categorical(np.array(y_test))
y_test_e=encoded_test

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
print(y_train_e.shape)
print(y_test_e.shape)

### **Model Building** 

We will be using two types of Deep Neural Networks:

- **ANN** (Artificial Neural Network - fully connected)
- **CNN** (Convolutional Neural Network)

#### **ANN**

In [ ]:
#Build the model
# 3 layers, 1 layer to flatten the image to a 28 x 28 = 784 vector
#           1 layer with 128 neurons and relu function
#           1 layer with 10 neurons and softmax function
#Create the neural network model
def create_model(): 
        model_ann = keras.Sequential([
            keras.layers.Flatten(input_shape=(128,128,3)),
            keras.layers.Dense(500,kernel_initializer='he_uniform', activation=tf.nn.relu),
            keras.layers.Dropout(0.01),
            keras.layers.Dense(700,kernel_initializer='he_uniform', activation=tf.nn.relu),
            keras.layers.Dropout(0.01),
            keras.layers.Dense(12, kernel_initializer='random_uniform',activation=tf.nn.softmax)
        ])
        #Compile the model
        #The loss function measures how well the model did on training , and then tries 
        #to improve on it using the optimizer
        model_ann.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model_ann

In [ ]:
model_ann=create_model()
model_ann.summary()

In [ ]:
#Train the model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model_ann.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
history=model_ann.fit(X_train, 
          y_train_e,  #It expects integers because of the sparse_categorical_crossentropy loss function
          epochs=200, #number of iterations over the entire dataset to train on
          batch_size=100,validation_split=0.2,callbacks=[es, mc],use_multiprocessing=True,verbose=1)#number of samples per gradient update for training

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
#Evaluate the model on test Data
model_ann.evaluate(X_test,y_test_e)

In [ ]:
from tensorflow.keras.models import load_model
model_ann=load_model('best_model_ann.h5')

In [ ]:
y_predict=model_ann.predict(X_test)
y_predict = np.argmax(y_predict, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test,y_predict)
cm

In [ ]:
plt.figure(figsize=(8,7))
import seaborn as sns
sns.heatmap(cm, annot=True,vmin=0, vmax=1, cmap='icefire_r')
plt.show()

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
test_df=pd.DataFrame({"y_test": y_test,
              "y_predict": y_predict
})

In [ ]:
test_df.head()

In [ ]:
fig=plt.figure(figsize=(10,4))
rand_img_num=np.random.randint(1,50)
test_img = test_df.iloc[rand_img_num]
fig.add_subplot(2,2,1)
plt.imshow(X_test[test_img['y_test']])
plt.title(classes[test_img['y_test']])
fig.add_subplot(2,2,2)
plt.imshow(X_test[test_img['y_predict']])
plt.title(classes[test_img['y_predict']])
plt.show()
print(test_img['y_test'])

In [ ]:
keras.backend.clear_session()

In [ ]:
plt.figure(figsize=(8,4))
ax=sns.stripplot(data=test_df,y='y_test', x='y_predict', jitter=True)
plt.title("Distribution of Prediction")
plt.ylim(0,12)
plt.grid(True)

As we see here, the **ANN does not show a good test accuracy**, since ANNs are unable to capture correlation characteristics of the image. 

**Let's try Convolutional Neural Networks, which take in the whole image as a 2D matrix instead.**

### **CNN**

**CNN with Dropout - Model 1**

In [ ]:
model_cnn_d = Sequential()
#

model_cnn_d.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (128,128,3)))
model_cnn_d.add(MaxPool2D(pool_size=(2,2)))
model_cnn_d.add(Dropout(0.25))
#
model_cnn_d.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_d.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_d.add(Dropout(0.25))
#
model_cnn_d.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_d.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_d.add(Dropout(0.3))
#
model_cnn_d.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model_cnn_d.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_d.add(Dropout(0.3))

#
model_cnn_d.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model_cnn_d.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_d.add(Dropout(0.3))

# 
model_cnn_d.add(Flatten())
model_cnn_d.add(Dense(1024, activation = "relu"))
model_cnn_d.add(Dropout(0.5))
model_cnn_d.add(Dense(12, activation = "softmax"))
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_cnn_d.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 200  
batch_size = 64



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
mc = ModelCheckpoint('best_model_cnn_d.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history_cnn_d=model_cnn_d.fit(X_train, 
          y_train_e,  #It expects integers because of the sparse_categorical_crossentropy loss function
          epochs=30, #number of iterations over the entire dataset to train on
          batch_size=64,validation_split=0.20,callbacks=[es, mc],use_multiprocessing=True,verbose=1)#number of samples per gradient update for training  


In [ ]:
# summarize history for accuracy
plt.plot(history_cnn_d.history['accuracy'])
plt.plot(history_cnn_d.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model_cnn_d=load_model('best_model_cnn_d.h5')

In [ ]:
#Evaluate the model on test Data
model_cnn_d.evaluate(X_test,y_test_e)

In [ ]:
y_predict=model_cnn_d.predict(X_test)
y_predict = np.argmax(y_predict, axis=1)

In [ ]:
cm=confusion_matrix(y_test,y_predict)
cm

In [ ]:
plt.figure(figsize=(8,7))
plt.title("Heatmap Test Data")
import seaborn as sns
sns.heatmap(cm, annot=True,vmin=0, vmax=1, cmap='icefire_r')
plt.show()

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
test_df_cnn_d=pd.DataFrame({"y_test": y_test,
              "y_predict": y_predict
})
test_df_cnn_d.head(5)

In [ ]:
fig=plt.figure(figsize=(10,4))
rand_img_num=np.random.randint(1,50)
test_img = test_df_cnn_d.iloc[rand_img_num]
fig.add_subplot(2,2,1)
plt.imshow(X_test[test_img['y_test']])
plt.title(classes[test_img['y_test']])
fig.add_subplot(2,2,2)
plt.imshow(X_test[test_img['y_predict']])
plt.title(classes[test_img['y_predict']])
plt.show()
print(test_img['y_test'])

In [ ]:
plt.figure(figsize=(8,4))
ax=sns.stripplot(data=test_df_cnn_d,y='y_test', x='y_predict', jitter=True)
plt.title("Distribution of Prediction")
plt.ylim(0,12)
plt.grid(True)

In [ ]:
keras.backend.clear_session()

**CNN with Dropout and BatchNormalization - Model 2**

In [ ]:


model_cnn_db = Sequential()
#
model_cnn_db.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (128,128,3)))
model_cnn_db.add(MaxPool2D(pool_size=(2,2)))
#model_cnn_db.add(BatchNormalization(axis=3))
model_cnn_db.add(Dropout(0.25))
#
#model_cnn_db.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_db.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model_cnn_db.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model_cnn_db.add(BatchNormalization(axis=3))
model_cnn_db.add(Dropout(0.25))
#
#model_cnn_db.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_db.add(Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model_cnn_db.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_db.add(BatchNormalization(axis=3))
model_cnn_db.add(Dropout(0.3))
#
#model_cnn_db.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same',  activation ='relu'))  
model_cnn_db.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',  activation ='relu'))                 
model_cnn_db.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
#model.add(BatchNormalization(axis=3))
model_cnn_db.add(Dropout(0.3))

#
#model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same',  activation ='relu'))
model_cnn_db.add(Conv2D(filters = 256, kernel_size = (3,3),padding = 'Same',  activation ='relu'))
model_cnn_db.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_db.add(BatchNormalization(axis=3))

# 
model_cnn_db.add(Flatten())
model_cnn_db.add(Dense(1024, activation = "relu"))
#model_cnn_db.add(Dropout(0.5))
model_cnn_db.add(Dense(12, activation = "softmax"))
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_cnn_db.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 200  
batch_size = 64



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
mc = ModelCheckpoint('best_model_db.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True,)
lrr = ReduceLROnPlateau(monitor='val_loss',  patience=3, verbose=1, factor=0.4, min_lr=0.00001)
lrr1 = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.1)

history_cnn_db=model_cnn_db.fit(X_train, 
          y_train_e,  #It expects integers because of the sparse_categorical_crossentropy loss function
          epochs=30, #number of iterations over the entire dataset to train on
          batch_size=64,
          validation_split=0.20,callbacks=[es, mc,],use_multiprocessing=True,verbose=1)#number of samples per gradient update for training  


In [ ]:

# summarize history for accuracy
plt.plot(history_cnn_db.history['accuracy'])
plt.plot(history_cnn_db.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model_cnn_db=load_model('best_model_db.h5')

In [ ]:
#Evaluate the model on test Data
model_cnn_db.evaluate(X_test,y_test_e)

In [ ]:
y_predict = model_cnn_db.predict(X_test)
y_predict = np.argmax(y_predict, axis=1)

In [ ]:
cm=confusion_matrix(y_test,y_predict)
cm

In [ ]:
plt.figure(figsize=(8,7))
plt.title("Heatmap Test Data")
import seaborn as sns
sns.heatmap(cm, annot=True,vmin=0, vmax=1, cmap='icefire_r')
plt.show()

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
test_df_db=pd.DataFrame({"y_test": y_test,
              "y_predict": y_predict
})
test_df_db.head(10)

In [ ]:
fig=plt.figure(figsize=(10,4))
rand_img_num=np.random.randint(1,50)
test_img = test_df_db.iloc[rand_img_num]
fig.add_subplot(2,2,1)
plt.imshow(X_test[test_img['y_test']])
plt.title(classes[test_img['y_test']])
fig.add_subplot(2,2,2)
plt.imshow(X_test[test_img['y_predict']])
plt.title(classes[test_img['y_predict']])
plt.show()
print(test_img['y_test'])

In [ ]:
plt.figure(figsize=(8,4))
ax=sns.stripplot(data=test_df_db,y='y_test', x='y_predict', jitter=True, dodge=True)
#ax=sns.violinplot(data=test_df_db,y='y_test', x='y_predict', jitter=True, dodge=True)
plt.title("Distribution of Prediction")
plt.ylim(0,15)
plt.grid(True)

**CNN with Dropout and Data Agumenation - Model 3**

In [ ]:
X_train_dg=X_train.copy()

In [ ]:
X_train_dg.shape

In [ ]:
train_datagen = ImageDataGenerator(rotation_range = 180,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,vertical_flip = True,width_shift_range = 0.1,height_shift_range = 0.1)


In [ ]:
train_datagen.fit(X_train_dg)

In [ ]:
model_cnn_da = Sequential()
#

model_cnn_da.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (128,128,3)))
model_cnn_da.add(MaxPool2D(pool_size=(2,2)))
model_cnn_da.add(Dropout(0.25))
#
model_cnn_da.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_da.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_da.add(Dropout(0.25))
#
model_cnn_da.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model_cnn_da.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_da.add(Dropout(0.3))
#
model_cnn_da.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model_cnn_da.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_da.add(Dropout(0.3))

#
model_cnn_da.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model_cnn_da.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model_cnn_da.add(Dropout(0.3))

# 
model_cnn_da.add(Flatten())
model_cnn_da.add(Dense(1024, activation = "relu"))
model_cnn_da.add(Dropout(0.5))
model_cnn_da.add(Dense(12, activation = "softmax"))
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
model_cnn_da.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
epochs = 200  
batch_size = 64



es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
mc = ModelCheckpoint('best_model_da.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history_cnn_da=model_cnn_da.fit(X_train_dg, 
          y_train_e,  #It expects integers because of the sparse_categorical_crossentropy loss function
          epochs=30, #number of iterations over the entire dataset to train on
          batch_size=64,validation_split=0.20,callbacks=[es, mc],use_multiprocessing=True,verbose=1)#number of samples per gradient update for training  

In [ ]:
# summarize history for accuracy
plt.plot(history_cnn_da.history['accuracy'])
plt.plot(history_cnn_da.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model_cnn_da=load_model('best_model_da.h5')

In [ ]:
#Evaluate the model on test Data
model_cnn_da.evaluate(X_test,y_test_e)

In [ ]:
y_predict = model_cnn_da.predict(X_test)
y_predict = np.argmax(y_predict, axis=1)

In [ ]:
cm=confusion_matrix(y_test,y_predict)
cm

In [ ]:
plt.figure(figsize=(8,7))
plt.title("Heatmap Test Data")
import seaborn as sns
sns.heatmap(cm, annot=True,vmin=0, vmax=1, cmap='icefire_r')
plt.show()

In [ ]:
print(classification_report(y_test,y_predict))

In [ ]:
test_df_da=pd.DataFrame({"y_test": y_test,
              "y_predict": y_predict
})
test_df_da.head(10)

In [ ]:
fig=plt.figure(figsize=(10,4))
rand_img_num=np.random.randint(1,50)
test_img = test_df_da.iloc[rand_img_num]
fig.add_subplot(2,2,1)
plt.imshow(X_test[test_img['y_test']])
plt.title(classes[test_img['y_test']])
fig.add_subplot(2,2,2)
plt.imshow(X_test[test_img['y_predict']])
plt.title(classes[test_img['y_predict']])
plt.show()
print(test_img['y_test'])

In [ ]:
xing=X_test*255

In [ ]:
plt.figure(figsize=(8,4))
ax=sns.stripplot(data=test_df_da,y='y_test', x='y_predict', jitter=True, dodge=True)
#ax=sns.violinplot(data=test_df_da,y='y_test', x='y_predict', jitter=True, dodge=True)
plt.title("Distribution of Prediction")
plt.ylim(0,15)
plt.grid(True)

In [ ]:
#%%shell
#jupyter nbconvert --to html '/content/drive/MyDrive/Colab Notebooks/plant-seedlings.ipynb'

In [ ]:
vgg_img=[]
for img in X_train:
  vggx_img = cv2.resize(img,(224,224))
  vgg_img.append(vggx_img)

In [ ]:
X_train.shape

In [ ]:
from keras.applications.vgg16 import VGG16
model_vgg = VGG16()
print(model_vgg.summary())

In [ ]:
y_predict_vgg=model_vgg.predict(X_test)